# ТТВ рейтинг в течение дня
Пример расчета показателей TTV в разбивке по часу.

## Описание задачи и условий расчета
- Период: 01.09.2022 - 30.09.2022 по рабочим и выходным отдельно
- Временной интервал: 05:00-29:00 с разбивкой по часу
- ЦА: Россия 100+
- Место просмотра: Все места (дом+дача)
- Каналы: Total Channels									
- Статистики: TTV Rtg%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# Найдем идентификаторы нужных нам типов дней
cats.get_tv_research_day_type()

Запрошены записи: 0 - 4
Всего найдено записей: 4



,id,name,ename
0,W,рабочий день,working day
1,H,праздничный день,public holiday
2,E,выходной день,weekend
3,A,день траура,day of mourning


In [3]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['100+'])

Запрошены записи: 0 - 2
Всего найдено записей: 2



,id,valueId,name,colName,valueName
0,118,1,Население 100+,Cube100Plus100Minus,100+
1,118,2,Население 100+,Cube100Plus100Minus,100-


## Формирование задания
Зададим условия расчета

In [4]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-09-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = 'researchDayType IN (W, E)'

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = 'cube100Plus100Minus = 1'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = None

# Указываем список статистик для расчета
statistics = ['TTVRtgPer']

# Указываем срезы
slices = ['researchDayType', 'timeBand60']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

## Расчет задания

In [5]:
# Формируем задание для TV API в формате JSON
task_json = mtask.build_task('timeband', date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df_timeband = mtask.result2table(mtask.get_result(task_timeband))

Расчет задачи (id: e2cf641b-926f-463e-b697-e2d008fa39fb) [= = = = = = = = ] время расчета: 0:00:30.968810


Настраиваем внешний вид таблицы

In [6]:
df_timeband = df_timeband.astype(dtype={'timeBand60':int}) #меняем формат для корректной сортировки

df_res = pd.pivot_table(df_timeband, values=['TTVRtgPer'],
                        index=['timeBand60'], 
                        columns=['researchDayType'])
df_res

TTVRtgPer           
researchDayType          E          W
timeBand60                           
50000             2.959879   3.012725
60000             4.022613   5.244354
70000             6.000277   7.460177
80000             9.203692   8.326921
90000            12.216830   9.771041
100000           14.475483  10.569377
110000           15.345393  11.477584
120000           15.861817  12.283134
130000           16.222483  12.647207
140000           16.869255  13.074837
150000           18.066413  13.977750
160000           19.153364  14.777870
170000           20.476772  16.801896
180000           23.280404  20.310693
190000           26.454624  23.714317
200000           28.556109  26.653814
210000           28.091255  26.599967
220000           24.260792  22.729935
230000           17.584367  16.270910
240000           11.455740   9.822293
250000            7.001630   5.877568
260000            4.403675   3.701053
270000            3.110648   2.780097
280000            2.652309   2.466464

## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [7]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('06_TTVRtg%_during_the_day'))
df_info = mtask.task_builder.get_report_info()
df_res.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()